In [37]:
#Libraries
import xarray as xr  
import numpy as np
import pandas as pd
from scipy import ndimage
from scipy.ndimage import convolve
import geopandas as gpd
from datetime import timedelta
import rioxarray
import rasterio
from geopy.distance import geodesic
import math
import sys
import os
import glob
import numbers as checknumbers
from shapely.geometry import MultiPolygon, Polygon, shape, Point, MultiPoint, mapping 
from shapely.wkt import loads
from shapely.ops import unary_union
import uuid
import tqdm
import time
import warnings
import copy
import cftime

warnings.filterwarnings("ignore")

In [38]:
UTM_WORLD_ZONE = 3857 
UTC_LOCAL_HOUR = 0


##----------------------------------------------------------Executing ATRACKCS for intercomparisson data
# pathResultados = sys.argv[1]
# path_obs = sys.argv[2] 
# run = sys.argv[3]

pathResultados = '/home/vrobledodelgado/hpchome/MCSMIP/AGAIN/OUTPUTS/WINTER/SCREAM/'
path_obs = '/home/vrobledodelgado/LSS/Humberto/Vanessa/'
run = 'model'
filename = 'olr_pcp_Winter_SCREAM.nc'
"""
Version 3:

Dates for winter: 2020-01-20 to 2020-02-28
Dates for summer: 2016-08-01 to 2016-09-10

Common MCS criteria: 
The criteria below are guidelines for defining an MCS and each tracker should 
try to implement them as closely as possible.
The area with Tb <= 241 K must be at least 40,000 km2  for at least 4 continuous
 hours

Minimum peak precipitation of 10 mm h-1 at least one pixel (0.1° x 0.1° for 1 hour)
 inside the whole MCS for 4 continuous hours
    Luego selecciono aquellos que tengan al meno 1 pixel con 10 mm/h en un pixel
    por al menos 4 horas

20,000 km2 mm h-1  (e.g., 100 km x 100 km x 2 mm h-1 ) minimum rainfall volume at least once in the lifetime of the MCS

"""

LOCATION_FOLIUM = [10, 135]
tb_value = 241
area_value = 5000
pp_rates = 4
variables = "Both"

tb_overshoot = 225
duration = 4
buffer_threshold = 0.1


In [39]:
#### ______________ reading data________________________________________________
def read_MCSMIP_data(path_data, run):
    """
    Function for reading Tb and P DataArrays in files. 
    The spatial resampling is 0.1° - lineal interpolation.
    The temporal resampling is 1 h - nearest original coordinate to up-sampled frequency coordinates.
    
    Inputs:
    pathTb: Path where the Tb raw data are located.
    pathP: Path where the P raw data are located.
    The path must have the next structure:
    linux: r"/home....../"
    windows: r"C:/....../"
    
    Outputs:
    Geodataframe with the polygons associated to the P and Tb spots.
    """
    #Reading P data
    ds_combined = xr.open_mfdataset(path_obs+filename)
    ds = ds_combined.rename(name_dict={'precipitation':'P'})

    if run == 'model':
        a = 1.228
        b = -1.106e-3
        sigma = 5.67e-8 # W m^-2 K^-4
        tf = (ds.olr/sigma)**0.25
        Tb = (-a + np.sqrt(a**2 + 4*b*tf))/(2*b)
        ds['olr'] = Tb
        ds = ds.rename(name_dict={'olr':'Tb'})
        try:
            ds = ds.drop_vars('xtime_bnds')
        except Exception as e:
            print("Exception:", e)
            pass
        try:
            ds = ds.drop_vars('time_bnds')
        except Exception as e:
            print("Exception:", e)
            pass

        #Converting the UTC to local hour
    datex = ds.time.coords.to_index()
    #Replacing the datetimeindex based on UTC_LOCAL_HOUR
    datedt = datex.to_pydatetime()
    dates_64 = [np.datetime64(row) for row in datedt]
    ds =  ds.assign_coords({"time": dates_64})
    
    #Extracting dimensiones: time, lat and lon
    dates = ds.time.values; 
    lon, lat = np.float32(np.meshgrid(ds.lon,ds.lat))
 
    #Establishing EPSG:4326 ACA hay que revisar que projection plana funciona para todo el globo
    ds = ds.rio.set_crs(4326)
    ds.attrs['crs'] = ds.rio.crs
    
    initial_date_lecture = str(ds.time[0].values)[:16]
    final_date_lecture = str(ds.time[-1].values)[:16]
    
    print('Complete data reading ' + initial_date_lecture + " - " + final_date_lecture)    
    return ds, dates, lon, lat

def polygon_identify(data_id, variable):
    """
    Function to draw polygons associated to each P or Tb spot; the polygon delimitation 
    is from the Convex Hull. This function execute inside the function identify_msc2. 
    
    Inputs:
    data_id = DataArray associated with each variable: P or Tb
    variable = string name variable: "Tb" or "P"
    
    Outputs:
    Geodataframe with the polygons associated to the P and tb spots.
    """
    #structure that determines which elements are neighbors to each pixel.
    structure = ndimage.generate_binary_structure(2,2)

    for step in data_id.time:
        # print(data_id)
        data_id_temp = data_id.sel(time = step)
        arr_filt = ndimage.binary_closing(data_id_temp.values, structure=np.ones((3,3))).astype(data_id_temp.values.dtype) ## llena huecos
        arr_filt = ndimage.binary_opening(arr_filt, structure=np.ones((3,3))).astype(arr_filt.dtype) ## borra outliers
        blobs, number_of_blobs = ndimage.label(arr_filt, structure=structure)
        distance = ndimage.distance_transform_edt(blobs)
    
        #only the outline of the polygons (spots) based on binary structure.
        distance[distance != 1] = 0
        blobs[distance == 0]= 0
        blobs = blobs.astype('float')
        #blobs[blobs  == 0] = np.nan
        
        #replacing the values generated in the original Datarray
        data_id.loc[dict(time = step)] = blobs
      
    data_id = data_id.where(data_id != 0, np.nan)         
    #___________________Polygon generation of the spots______________________________
        
    #Datarray to Dataframe
    df = data_id.to_dataframe().reset_index()
    df.dropna(inplace = True)
    #Dataframe to Geodataframe
    gdf = gpd.GeoDataFrame(
            df[["time", variable]], geometry=gpd.points_from_xy(df.lon,df.lat))
     
    #Extracting the coordinates of the georeferenced points in a separate column
    gdf['geo'] = gdf['geometry'].apply(lambda x: x.coords[0])
    
    #Converting the points-groups to polygons based on the Convex Hull
    df = gdf.groupby(by=["time" , variable])['geo'].apply(lambda x: MultiPoint(sorted(x.tolist()))).to_frame().reset_index()
    df["geo"] = df["geo"].apply(lambda x: x.convex_hull.wkt).to_frame()
    df["geo"] = df["geo"].apply(lambda x: loads(x)).to_frame()
    
    #Making the Geodataframe final 
    gdf = gpd.GeoDataFrame(df, geometry=df.geo, crs=4326); del df; del gdf["geo"]
    #Dropping lines and solitary points from the Geodataframe
    gdf = gdf.loc[gdf['geometry'].geom_type=='Polygon']
         
    return gdf 
    
def merge_neighbours(gdf_tb,ds,buffer_threshold):
    ##create a empy gdf 
    gdf_merged = gpd.GeoDataFrame(columns=["time", "geometry"], crs="EPSG:4326")

    for step in ds.time.values:
        test = gdf_tb[gdf_tb['time']== step]
        
        proximity_threshold = buffer_threshold
        test['buffered'] = test.geometry.buffer(proximity_threshold)
        
        merged_geometry = unary_union(test['buffered'])
        merged_polygons = [polygon for polygon in merged_geometry.geoms] if merged_geometry.geom_type == 'MultiPolygon' else [merged_geometry]
        
        merged_gdf = gpd.GeoDataFrame(geometry=merged_polygons, crs=test.crs)
        merged_gdf['time'] = step
        gdf_merged = pd.concat([gdf_merged, merged_gdf], ignore_index=True)
    
    gdf_merged = gdf_merged.reset_index(drop=True)
    gdf_merged['Tb'] = np.arange(1,len(gdf_merged)+1)
    
    return gdf_merged

def identify_msc2(data, variables,Tb, area_Tb, pp_rates, buffer_threshold):
    """
    Function for identifying the spots, in a time step, based on the methodology
    of brightness temperature and precipitation association.
 
    Inputs
    data = DataArray of the variables
    variables: Methodology of association "Tb" or "Both":Tb and P. 
    If "Tb" the spots area are delimited by brightness temperature cold cloud top
    Elif "Both" the spots area are delimited by brightness temperature cold cloud top and
    precipitation threshold in each spot
    pp_rates = precipitation threshold


    #default parameters based on literature
    Tb (Brightness Temperature): spots based on limited threshold cold cloud top (ex. <225 K)[Feng et al.,(2021); Li et al.,(2020)]
    area_Tb: spots with a minimun largest area polygon (ex. > 2000 km2)[Lui et al., (2019); Vizy & Cook,(2018)] 
    P (Precipitation): By default is the spots greater than 2 mm/h with length[Feng et al.,(2021)].         

    Outputs:
    Geodataframe with the polygons associated to the methodology choosed
    """ 
    kernel = np.ones((5, 5))    
    global counter, storms_counter
    if variables == "Both":
        # Masking the DataArray based on limited threshold cold cloud top
        mask_tb = (data["Tb"] <= Tb).astype(int)
        
        # Convert mask to DataArray and establish CRS
        datax = mask_tb.to_dataset(name="Tb")
        datax = datax.rio.write_crs(4326)
    
    #selecting Tb variable
    datax = datax['Tb']
    datax[:,0, :] = 0          # First row
    datax[:,-1, :] = 0         # Last row
    datax[:,:, 0] = 0          # First column
    datax[:,:, -1] = 0 
    
    data_p = data['P']
    #Identifying polygons of Tb and P of the original dataArray  
    gdf_tb = polygon_identify(datax, variable = "Tb")

    gdf_tb2 = merge_neighbours(gdf_tb,datax,buffer_threshold)

    #Establishing world mercator
    gdf = gdf_tb2.to_crs(UTM_WORLD_ZONE) 
    
    #Calculating polygon area   
    gdf["area_tb"] = gdf['geometry'].area #meters
    gdf["area_tb"] = gdf["area_tb"]/10**6 #meters to kilometers
    gdf["area_tb"] = gdf["area_tb"].round(1)
    
    #Dropping polygons with an area less than area_Tb
    gdf = gdf.loc[gdf['area_tb'] >= area_Tb]    
    
    #Calculating centroids
    gdf["centroides"] = gdf.geometry.centroid
    gdf.reset_index(inplace = True, drop = True)
        
    print("Spots identification completed")

    return gdf

def clip_tb(sup, ds,tb_overshoot):
    """
    Function for estimating max, mean and min brightness temperature of each spot 
    
    Inputs
    sup: GeodataFrame containing the Tb polygons generated in the process "identify_msc2".
    ds = DataArray associated with the variables P and Tb resulting from the process 'read_resample_data'.
    
    Outputs:
    GeoDataFrame of the identified polygons with the Tb features
    """
    #Preparing the DataArray
    data_magnitud = ds.copy()
    data_magnitud = data_magnitud.to_dataframe()
    data_magnitud_ = data_magnitud.to_xarray()["Tb"]
    
    #Establishing EPSG:4326 - WGS geodetic coordinate system 
    data_magnitud_ = data_magnitud_.rio.write_crs(4326)  
    
    #Renaming dimensions 
    data_magnitud_= data_magnitud_.rename({"lat":"y", "lon":"x"})
    gdf_tb = sup['geometry']

    #Creating new columns polygons with tb 225 
    sup['tb_225'] = None

    print ("Estimating Tb spots features: ")    
    #Estimating Tb features
    for index_t,_dates, progress in zip(gdf_tb.index,sup.time, tqdm.tqdm(range(len(sup.time)))):
        _polygon = gdf_tb.geometry.loc[index_t]
        coordinates = np.dstack((_polygon.exterior.coords.xy[0], _polygon.exterior.xy[1]))
        geometries = [{'type': 'Polygon', 'coordinates': [coordinates[0]]}]    
        blob_clipped = data_magnitud_.sel(time=_dates).rio.clip(geometries, gdf_tb.crs, drop=False, invert=False)
        
        blob_clipped2 = blob_clipped.where(blob_clipped <= tb_overshoot) 
        #Checking if the DataArray has at least 1 pixels with tb overshoot.
        if blob_clipped2.notnull().sum() >= 1:
            overshoot = True
        else:
            overshoot = False

        sup.loc[index_t,"tb_225"] = overshoot

        #Progress bar
        time.sleep(0.01)
        
    return sup

def clip_tb_pp22(sup, ds, pp_rates, drop_empty_precipitation=True):
    """
    Updated function with error handling and performance improvements.
    """
    # Prepare DataArray
    data_magnitud = ds.copy()
    #data_magnitud = data_magnitud.chunk({'time': 1, 'lat': 100, 'lon': 100})  # Add lazy loading
    data_magnitud = data_magnitud.to_dataframe()
    data_magnitud_ = data_magnitud.to_xarray()["P"]
    
    #Establishing EPSG:4326 - WGS geodetic coordinate system 
    
    #data_magnitud_ = data_magnitud["P"].rio.write_crs(4326)  
    data_magnitud_ = data_magnitud_.rio.write_crs(4326) 
    
    #Renaming dimensions 
    data_magnitud_= data_magnitud_.rename({"lat":"y", "lon":"x"}) 
    gdf_tb = sup['geometry']
    
    # Initialize columns
    sup['pp_10rate'] = None
    sup['volum_pp'] = None
    sup['mean_pp'] = None
    
    print("Estimating P spots features: ")

    for index_t,_dates, progress in zip(gdf_tb.index,sup.time,tqdm.tqdm(range(len(sup.time)))):
        _polygon = gdf_tb.geometry.loc[index_t]
        coordinates = np.dstack((_polygon.exterior.coords.xy[0], _polygon.exterior.xy[1]))
        geometries = [{'type': 'Polygon', 'coordinates': [coordinates[0]]}]

        #Applying criteria up to 2 mm/h
        blob_clipped = data_magnitud_.sel(time=_dates).rio.clip(geometries, gdf_tb.crs, drop=False, invert=False)
        blob_clipped = blob_clipped.where(blob_clipped >= pp_rates)
        #Checking if the DataArray has at least 5 pixels with precipitation.
        if blob_clipped.notnull().sum() >= 5:
            mean_magnitud = round(float(blob_clipped.mean(skipna=True).values),4)
        else:
            mean_magnitud = np.nan
        
        #Checking if the DataArray has at least 100km*100km (10 pixels) with precipitation > 2mm/h.
        blob_clipped1 = blob_clipped.where(blob_clipped >= 2) 
        if blob_clipped1.notnull().sum() >= 10:
            volume_ppt = True
        else:
            volume_ppt = False
            
        #Checking if the DataArray has at least one pixel  with precipitation > 10mm/h
        blob_clipped2 = blob_clipped.where(blob_clipped >= 10) 
        #Checking if the DataArray has at least 1 pixels with precipitation.
        if blob_clipped2.notnull().sum() >= 1:
            pixels_10mm = True
        else:
            pixels_10mm = False
            
        sup.loc[index_t,'volum_pp'] = volume_ppt
        sup.loc[index_t,'pp_10rate'] = pixels_10mm
        sup.loc[index_t,"mean_pp"] = mean_magnitud
        #sup['pp_10rate'] = round(sup['pp_10rate'].astype(float),4)
        
        #Progress bar
        time.sleep(0.01)       
        
        #Dropping polygons that do not meet the precipitation parameters
        #in terms of area and amount.
    if drop_empty_precipitation == True:
        sup = sup[sup['mean_pp'].notna()].reset_index(drop=True)
    else: 
        pass
    return sup


def min_dist(point, gpd2):
    """
    Function to find the nearest polygon based on a georeferenced point.
    
    Inputs:
    point: Georeferenced point
    gpd2: Geodataframe - Polygons in a specific time and region
    
    Outputs:
    Geoseries with the nearest polygon to the georeferenced point.
    """
    gpd = gpd2.copy()
    gpd['Dist'] = gpd.apply(lambda row:  point.distance(row.geometry),axis=1)
    geoseries = gpd.iloc[gpd['Dist'].argmin()]
    return geoseries


def finder_msc2(sup, threshold_overlapping_percentage = None):
    global msc_counter
    """
    Function for tracking convective systems according to the threshold_overlapping_percentage.
    This functions works based on identified convective systems in a period of time.   

    Inputs:
    sup: GeodataFrame generated in the process "identify_msc2"
    threshold_overlapping_percentage (float): By default there is no percentage overlap limit 
    between polygons, however, this can be set. 
    (ex. 10: This means that the percentage of overlap that is not greater than or equal to 10%, 
    in reference to the largest area polygon, is not taken into account in the trajectory).
    
    Outputs:
    GeodataFrame which identifies the tracks in the time period of interest
    """
    msc_counter = 1
    pd.options.mode.chained_assignment = None
    range_sites = sup.index
    #Making column belong
    sup.loc[sup.index, "belong"] = 0
    #Making column percentage intersection
    sup.loc[sup.index, "intersection_percentage"] = np.nan
        
    #Generating tracks
    
    print ("Estimating trajectories: " )
    for isites, progress in zip(range_sites, tqdm.tqdm(range(len(range_sites)))):
        # Checking when the record (spot) has not yet been processed or attaching to a track
        if sup.at[isites, "belong"] == 0:
            try:
                #Assigning the counter number to identify the track(range 1) 
                sup.at[isites, "belong"] = msc_counter #Index, col  
                
                #Generating start and end time date (time t and time t+1)
                date_initial = sup.loc[isites].time
                date_final = date_initial + timedelta(hours=1)
                
                #Reference spot
                poli_ti = sup.loc[isites].to_frame().T
                #Establishing EPSG:32718 - UTM zone 18S plane coordinate system 
                poli_ti = gpd.GeoDataFrame(poli_ti, geometry='geometry', crs=UTM_WORLD_ZONE)
                
                #Spots one hour later
                poli_ti1 = sup.loc[sup.time == date_final]
                
                #Intersection of reference spot with spots one hour later
                intersect = gpd.overlay(poli_ti, poli_ti1, how='intersection') ##cogee el politi y muestra con cuales del siguiente paso se traslapan 
                                                      
                #Nearest intersection point (centroid)
                
                try: #When there is only one intersection spot
                    
                    #Estimating the distance between centroids
                    dist = min_dist(intersect.centroid, poli_ti1)
                                
                    #Extracting the spot with the maxima area
                    intersect_area_max = max(intersect.area_tb_1.values, intersect.area_tb_2.values)
    
                    #Estimating percentage of intersection
                    intersect_percentage = ((intersect.area/1000000)*100/intersect_area_max)[0].round(1)                
                    
                    #Condition if threshold overlapping percentage is activated
                    #If the spot fails with threshold overlapping percentage is dropped
                    if isinstance(threshold_overlapping_percentage, checknumbers.Real):
                        if intersect_percentage>= threshold_overlapping_percentage:
                            sup.at[isites, "intersection_percentage"] = intersect_percentage #Index, col 
                            #print ("There was a " + str(intersect_percentage) + "% intersection") 

                        else:
                            sup.drop(isites, inplace = True)
                            #print ("The convective system " + str(isites) + " was dropped to the data due intersection is below that the threshold: "+ str(intersect_percentage)) 

                    #Condition if threshold overlapping percentage is not activated
                    else:
                        sup.at[isites, "intersection_percentage"] = intersect_percentage #Index, col 
                        #print ("There was a " + str(intersect_percentage) + "% intersection") 

                except: #When there are more than one intersection spot is selected the bigger spot / select the nearest spot
                    
                    #dimensioning the Geodataframe columns only for the intersected spots
                    intersect = intersect[intersect.columns[10:]]  #poligonos con los que se intersecta
                    
                    #Extracting the intersection spot with the maxima area of intersection
                    intersect['area_intersected'] = intersect.area/10**6
                    
                    a_int = intersect.loc[:, intersect.columns.str.contains('area_intersected')].idxmax().values[0] 
                    a_pol = intersect.loc[:, intersect.columns.str.contains('area_tb')].idxmax().values[0] 
                    if a_int == a_pol:
                        bigger_polygon = a_int
                    else:
                        bigger_polygon = a_pol
                    
                    #Preparing Geodataframe - renaming columns
                    interest_polygon = intersect.loc[bigger_polygon].to_frame()
                    name_columns = ['intersection_percentage', 'time', 'Tb', 'area_tb', 
                                    'centroides', 'mean_tb', 'min_tb', 'max_tb', 'mean_pp','max_pp', 'belong', 
                                    'intersection_percentage', 'geometry','area_intersected']
                    interest_polygon_t = interest_polygon.T ##este es el poligono de la interseccion no el siguiente
                    interest_polygon_t.columns = name_columns
                    covex = gpd.GeoDataFrame(interest_polygon_t, geometry=interest_polygon_t.geometry)

                    #Area of the bigger spot                
                    area_bigger_polygon = covex.area_tb.values[0]
                    
        
                    #Estimating the distance between centroids  ???               
                    dist = min_dist(covex.centroid, poli_ti1)
                                        
                    #Estimating percentage of intersection between t0 and t+1               
                    intersect_percentage = ((covex['area_intersected'].values[0])*100/area_bigger_polygon).round(1)                

                    #Condition if threshold overlapping percentage is activated
                    #If the spot fails with threshold overlapping percentage is dropped                    
                    if isinstance(threshold_overlapping_percentage, checknumbers.Real):
                        if intersect_percentage>= threshold_overlapping_percentage:
                            sup.at[isites, "intersection_percentage"] = intersect_percentage 
                            #print ("There was a " + str(intersect_percentage) + "% intersection") 
                        else:
                            sup.drop(isites, inplace = True)
                            #print ("The convective system " + str(isites) + " was dropped to the data due intersection is below that the threshold: "+ str(intersect_percentage)) 
                    #Condition if threshold overlapping percentage is not activated
                    else:
                        sup.at[isites, "intersection_percentage"] = intersect_percentage 
                        #print ("There was a " + str(intersect_percentage) + "% intersection") 
                    #print ("Warning: in the tracking of this convective system were more than 1 intersecting polygon: "  + str(isites))

                #Attaching found spot to the track
                time_intersect = dist.to_frame().T["time"].values[0]
                tb_intersect = dist.to_frame().T["Tb"].values[0]
                sup.loc[(sup.time == time_intersect) & (sup.Tb == tb_intersect), "belong"] = msc_counter
                msc_counter +=1
                #print ("The convective system: " + str(isites) + " - belongs to the track: " +str(int(msc_counter)))
            except:
               msc_counter +=1

        # Checking when the record (spot) has been processed or attaching to a track            
        elif sup.at[isites, "belong"] in range(1, msc_counter):
            try:
                #Generating start and end time date (time t and time t+1)                
                date_initial = sup.iloc[isites].time
                date_final = date_initial + timedelta(hours=1)
            
                #Reference spot
                poli_ti = sup.loc[isites].to_frame().T
                #Establishing EPSG:32718 - UTM zone 18S plane coordinate system                 
                poli_ti = gpd.GeoDataFrame(poli_ti, geometry='geometry', crs=UTM_WORLD_ZONE)
                #Getting the id track previously estimate
                index_track = poli_ti.belong
                
                #Spots one hour later
                poli_ti1 = sup.loc[(sup.time == date_final) & (sup.belong == 0)]
    
                #Intersection of reference spot with spots one hour later
                intersect = gpd.overlay(poli_ti, poli_ti1, how='intersection')
                
                try:#When there is only one intersection spot
                    
                    #Estimating the distance between centroids
                    dist = min_dist(intersect.centroid, poli_ti1)
                    
                    #Extracting the spot with the maxima area
                    intersect_area_max = max(intersect.area_tb_1.values, intersect.area_tb_2.values)
    
                    #Estimating percentage of intersection                    
                    intersect_percentage = ((intersect.area/1000000)*100/intersect_area_max)[0].round(1)                
                    #print ("There was a " + str(intersect_percentage) + "% intersection") 
                    
                    #Condition if threshold overlapping percentage is activated
                    #If the spot fails with threshold overlapping percentage is dropped
                    if isinstance(threshold_overlapping_percentage, checknumbers.Real):
                        if intersect_percentage>= threshold_overlapping_percentage:
                            sup.at[isites, "intersection_percentage"] = intersect_percentage 
                            #print ("There was a " + str(intersect_percentage) + "% intersection") 
                        else:
                            sup.drop(isites, inplace = True)
                            #print ("The convective system " + str(isites) + " was dropped to the data due intersection is below that the threshold: "+ str(intersect_percentage)) 
                    
                    #Condition if threshold overlapping percentage is not activated
                    else:
                        sup.at[isites, "intersection_percentage"] = intersect_percentage 
                        #print ("There was a " + str(intersect_percentage) + "% intersection") 

                except:  #When there are more than one intersection spot is selected the bigger spot
                   
                    #dimensioning the Geodataframe columns only for the intersected spots
                    intersect = intersect[intersect.columns[10:]]
                    
                    #Extracting the spot with the maxima area
                    intersect['area_intersected'] = intersect.area/10**6
                    
                    a_int = intersect.loc[:, intersect.columns.str.contains('area_intersected')].idxmax().values[0] 
                    a_pol = intersect.loc[:, intersect.columns.str.contains('area_tb')].idxmax().values[0] 
                    if a_int == a_pol:
                        bigger_polygon = a_int
                    else:
                        bigger_polygon = a_pol 

                                        
                    #Preparing Geodataframe - renaming columns                    
                    interest_polygon = intersect.loc[bigger_polygon].to_frame()
                    name_columns = ['intersection_percentage', 'time', 'Tb', 'area_tb', 
                                    'centroides', 'mean_tb', 'min_tb', 'max_tb', 'mean_pp','max_pp', 'belong', 
                                    'intersection_percentage', 'geometry', 'area_intersected']
                    interest_polygon_t = interest_polygon.T
                    interest_polygon_t.columns = name_columns
                    
                    #Area of the bigger spot                
                    covex = gpd.GeoDataFrame(interest_polygon_t, geometry=interest_polygon_t.geometry)

                    #Area of the bigger spot                
                    area_bigger_polygon = covex.area_tb.values[0]

                    #Estimating the distance between centroids                                                                                            
                    #intersect = intersect.loc[bigger_polygon].to_frame().T  #interest_polygon.T
                    dist = min_dist(covex.centroid, poli_ti1)
                
                    #Estimating percentage of intersection                    
                    intersect_percentage = ((covex['area_intersected'].values[0])*100/area_bigger_polygon).round(1)                 

                    #Condition if threshold overlapping percentage is activated
                    #If the spot fails with threshold overlapping percentage is dropped                    
                    if isinstance(threshold_overlapping_percentage, checknumbers.Real):
                        if intersect_percentage>= threshold_overlapping_percentage:
                            sup.at[isites, "intersection_percentage"] = intersect_percentage 
                            #print ("There was a " + str(intersect_percentage) + "% intersection") 
                        else:
                            sup.drop(isites, inplace = True)
                            #print ("The convective system " + str(isites) + " was dropped to the data due intersection is below that the threshold: "+ str(intersect_percentage)) 
                    #Condition if threshold overlapping percentage is not activated
                    else:
                        sup.at[isites, "intersection_percentage"] = intersect_percentage 
                        #print ("There was a " + str(intersect_percentage) + "% intersection") 
                #print ("Warning: in the tracking of this convective system were more than 1 intersecting polygon: "  + str(isites))

                #Attaching found spot to the track
                time_intersect = dist.to_frame().T["time"].values[0]
                tb_intersect = dist.to_frame().T["Tb"].values[0]
                sup.loc[(sup.time == time_intersect) & (sup.Tb == tb_intersect), "belong"] = index_track.values[0]
                #print ("The convective system: " + str(isites) + " - belongs to the track: " +str(int(index_track.values[0])))
            except:
                pass   
            
        #Progress bar
        time.sleep(0.01)    
       
    #Transforming plane coordinates to geodesics  coordinates   
    sup["centroides"] = sup["centroides"].to_crs(4326)
    sup = sup.to_crs(4326) 
    
    #Creating an original index
    sup["id_gdf"] = None
    sup["id_gdf"] = sup.index
    return sup
    
def resume_track(sup, initial_time_hour):
    """
    Function for calculating characteristics associated with each tracking
    average speed, average distance, average direction, 
    total duration, average area, total distance traveled, total time traveled
    
    Inputs:
    sup: DataFrame generated in process "finder_msc2"
    initial_time_hour: Default is 0, but could chnage based on in a specific hour duration tracks
    
    Outputs:
    DataFrame containing tracks and features
    """      

    #Preparing Dataframe   
    NUEVODF = sup.copy()
    NUEVODF = NUEVODF.set_index(['belong', 'id_gdf']).sort_index()

    #Replacing old id for spots and tracks based on alphanumeric code 16 and 20
    #characteres respectively    
    new_belong = []
    new_track_id = []
    
    for track_id in NUEVODF.index.levels[1]:
        new_track_id.append(str(uuid.uuid4())[-22:])
    dic_replace_track_id = dict(zip(NUEVODF.index.levels[1].values, new_track_id))
    
    for belong_id in NUEVODF.index.levels[0]:
        new_belong.append(str(uuid.uuid4())[:13])
    dic_replace_belong = dict(zip(NUEVODF.index.levels[0].values, new_belong))
    
    reg_sup_res =  NUEVODF.reset_index()
    
    reg_sup_res.belong = reg_sup_res.belong.replace(dic_replace_belong)
    reg_sup_res.id_gdf = reg_sup_res.id_gdf.replace(dic_replace_track_id)
       
    reg_sup_res = reg_sup_res.drop(labels=['Tb'],axis=1)
       
    reg_sup = reg_sup_res.set_index(["belong" , "id_gdf"]).sort_index()
    
    #Attaching to the dataframe total duration and total distance for each spot. Each spot 
    #has the register of the features of his own track
    reg_sup["total_duration"] = None
    #reg_sup["total_distance"] = None
    
    count_df = reg_sup_res.groupby(by = [reg_sup_res.belong]).count()
    #sum_df = reg_sup_res.groupby(by = [reg_sup_res.belong]).sum()
    
    for _b in count_df.index: 
    
        count_value = count_df.loc[_b,"id_gdf"]
        #sum_value = sum_df.loc[_b,"distancia"]

        reg_sup.loc[_b, "total_duration"] = count_value
        #reg_sup.loc[_b, "total_distance"] = sum_value

    #Estimating track mean velocity      
    reg_deep_convection = reg_sup[reg_sup['total_duration'] >= initial_time_hour]

    #extraigo tambien el original 
    reg_deep_convection['time'] = pd.to_datetime(reg_deep_convection['time'], format="%Y-%m-%d %H:%M:%S")
    groupbelong2 = reg_deep_convection.groupby(level=[0])
    sorttime_deep_convect = groupbelong2.apply(lambda x: x.sort_values(by='time'))
    sorttime_deep_convect = sorttime_deep_convect[['time', 'geometry', 'area_tb', 'centroides', 'tb_225', 'pp_10rate','volum_pp','total_duration']]
    sorttime_deep_convect =  sorttime_deep_convect.droplevel(0)
    
    #Saving as .csv the resultsfor MCS
    # sorttime_reg_sup.to_csv(pathResultados + "resume_MCS2_"+str(reg_sup.time.min())[:-7]+"_"+str(reg_sup.time.max())[:-7]+".csv")
    sorttime_deep_convect.to_csv(pathResultados + "resume_DeepConvection2_"+str(reg_sup.time.min())[:-7]+"_"+str(reg_sup.time.max())[:-7]+".csv")
 
    return sorttime_deep_convect

def plot_folium(resume, location, path_save):
    """
    function for plotting tracks results in folium map. 

    Inputs:
    * resume: GeoDataFrame, data related with the tracks and MCS's.
    * location list (lat, lon), location for center the map_folium.
    * path_save: str, path where the .html folium map will be saved   
    
    Outputs:
    * the .html folium map will be open with the librarie "webbrowser"
    * path_saved: str, path where the .html was saved.
    """
    m = folium.Map(location=location, zoom_start=5, tiles='CartoDB positron')
    
    df = resume.reset_index()
    
    for i in df.belong.unique():
        #Sorting index by time
        tracks = df.loc[df.belong == i].reset_index()
        tracks  = tracks.set_index("time").sort_index()
        tracks = tracks.reset_index()
        for idn, r in tracks.iterrows():

            sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
            geo_j = sim_geo.to_json()
            geo_j = folium.GeoJson(data=geo_j,
                                  style_function=lambda x: {'fillColor': 'orange'})
            folium.Popup(r.index).add_to(geo_j)
            try: #Tb and P methodlogy
                folium.Marker(location=[r['centroides'].y, r['centroides'].x], popup='id_track: {} <br> id_mcs: {} <br> hour_mcs: {} <br> time: {} <br> area[km2]: {} <br> total_duration[h]: {} <br>'.format(r['belong'], r["id_gdf"], idn, r["time"], round(r['area_tb'],1), r["total_duration"])).add_to(m)
                extra_name = "Tb_P_"
            except: #Tb methodlogy
                folium.Marker(location=[r['centroides'].y, r['centroides'].x], popup='id_track: {} <br> id_mcs: {} <br> hour_mcs: {} <br> time: {} <br> area[km2]: {} <br> total_duration[h]: {} <br>'.format(r['belong'], r["id_gdf"], idn, r["time"], round(r['area_tb'],1), r["total_duration"])).add_to(m)            
                extra_name = "Tb_"
            geo_j.add_to(m)
        
    min_time = str(resume.time.min())[:-6].replace("-","_").replace(" ","_")
    max_time = str(resume.time.max())[:-6].replace("-","_").replace(" ","_")
    path_result = path_save+'map_'+extra_name+min_time+"_"+max_time+".html"
    m.save(path_result)
    try:
        webbrowser.open(path_result)
    except:
        pass
    
    return path_result

In [40]:
#reading dataset
ds, dates, lon, lat = read_MCSMIP_data(path_obs,run) 


Se produjo una excepción: These variables cannot be found in this dataset: ['xtime_bnds']
Complete data reading 2020-01-20T00:00 - 2020-02-28T23:00


In [41]:
#Spots identificaction
sup = identify_msc2(ds, variables, tb_value, area_value, pp_rates,buffer_threshold)   


Spots identification completed


In [42]:
#Estimating brightness temperature features ()
sup = clip_tb(sup, ds, tb_overshoot)


Estimating Tb spots features: 


100%|█████████▉| 60302/60303 [44:12<00:00, 22.73it/s]  


In [43]:
#Estimating precipitation features. The criteria is based on the variables methodology used in identify_msc2.
#Coninue precipitation
sup = clip_tb_pp22(sup, ds, pp_rates, drop_empty_precipitation = True) 

Estimating P spots features: 


100%|█████████▉| 60302/60303 [1:08:03<00:00, 14.77it/s] 


In [44]:
#Tracks identification
sup = finder_msc2(sup, threshold_overlapping_percentage = None) 


Estimating trajectories: 


100%|█████████▉| 46695/46696 [29:46<00:00, 26.14it/s]


In [45]:
#Resume track
deep_convection = resume_track(sup, duration)

In [10]:
# import webbrowser
# import folium
# path_html_folium = plot_folium(deep_convection, location = LOCATION_FOLIUM, path_save = pathResultados)